In [7]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings("ignore")

In [8]:
X,y = make_classification(n_samples=1000, n_features=10, n_informative=2, n_redundant=8,
                          weights=[0.9,0.1] ,flip_y=0, random_state=42)
np.unique(y,return_counts=True)

(array([0, 1]), array([900, 100], dtype=int64))

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y,random_state=42)

In [10]:
params = {'solver': 'lbfgs',
        'max_iter': 1000, 
        'multi_class': 'auto',
        'random_state': 8888,}

lr = LogisticRegression(**params)
lr.fit(X_train, y_train)

y_pred = lr.predict(X_test)

report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.95      0.97      0.96       270
           1       0.62      0.50      0.56        30

    accuracy                           0.92       300
   macro avg       0.79      0.73      0.76       300
weighted avg       0.91      0.92      0.92       300



In [11]:
report_dict = classification_report(y_test, y_pred, output_dict=True)
report_dict

{'0': {'precision': 0.9456521739130435,
  'recall': 0.9666666666666667,
  'f1-score': 0.9560439560439561,
  'support': 270.0},
 '1': {'precision': 0.625,
  'recall': 0.5,
  'f1-score': 0.5555555555555556,
  'support': 30.0},
 'accuracy': 0.92,
 'macro avg': {'precision': 0.7853260869565217,
  'recall': 0.7333333333333334,
  'f1-score': 0.7557997557997558,
  'support': 300.0},
 'weighted avg': {'precision': 0.9135869565217392,
  'recall': 0.92,
  'f1-score': 0.9159951159951161,
  'support': 300.0}}

In [12]:
import mlflow


In [33]:
mlflow.set_tracking_uri("http://127.0.0.1:5000/")
mlflow.set_experiment('First Experiment')
# mlflow.set_tracking_uri("http://127.0.0.1:5000/")

with mlflow.start_run():
    mlflow.log_params(params)
    mlflow.log_metrics({
        'accuracy': report_dict['accuracy'],
        'recell_class_0': report_dict['0']['recall'],
        'recell_class_1': report_dict['1']['recall'],
        'f1_score_macro': report_dict['macro avg']['f1-score']
    })
    # mlflow.log_artifact('classification_report.txt')
    mlflow.sklearn.log_model(lr, 'Logistic Regression')
    # mlflow.log_artifact('model.pkl')

2025/01/30 15:12:12 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run bustling-gull-288 at: http://127.0.0.1:5000/#/experiments/875031596032780637/runs/567dd38fbedd4830bedf1b80ea4dc3b1
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/875031596032780637


In [15]:
report_dict['macro avg']['f1-score']

0.7557997557997558

In [34]:
log_reg = LogisticRegression(C=1,solver='liblinear')
log_reg.fit(X_train, y_train)
y_pred_log_reg = log_reg.predict(X_test)
print(classification_report(y_test, y_pred_log_reg))

              precision    recall  f1-score   support

           0       0.95      0.96      0.95       270
           1       0.60      0.50      0.55        30

    accuracy                           0.92       300
   macro avg       0.77      0.73      0.75       300
weighted avg       0.91      0.92      0.91       300



In [35]:
rf_clf = RandomForestClassifier(n_estimators=30, max_depth=3)
rf_clf.fit(X_train, y_train)
y_pred_rf = rf_clf.predict(X_test)
print(classification_report(y_test, y_pred_rf))

              precision    recall  f1-score   support

           0       0.96      1.00      0.98       270
           1       0.95      0.67      0.78        30

    accuracy                           0.96       300
   macro avg       0.96      0.83      0.88       300
weighted avg       0.96      0.96      0.96       300



In [36]:
xgb_clf = XGBClassifier( use_label_encoder = False , eval_metric='logloss')
xgb_clf.fit(X_train, y_train)
y_pred_xgb = xgb_clf.predict(X_test)
print(classification_report(y_test, y_pred_xgb))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99       270
           1       0.96      0.80      0.87        30

    accuracy                           0.98       300
   macro avg       0.97      0.90      0.93       300
weighted avg       0.98      0.98      0.98       300



In [44]:
from imblearn.combine import SMOTETomek
smt = SMOTETomek(random_state=42)
X_train_res, y_train_res = smt.fit_resample(X_train, y_train)

np.unique(y_train, return_counts=True)

(array([0, 1]), array([630,  70], dtype=int64))

In [45]:
xgb_clf = XGBClassifier( use_label_encoder = False , eval_metric='logloss')
xgb_clf.fit(X_train_res, y_train_res)
y_pred_xgb = xgb_clf.predict(X_test)
print(classification_report(y_test, y_pred_xgb))

              precision    recall  f1-score   support

           0       0.98      0.98      0.98       270
           1       0.81      0.83      0.82        30

    accuracy                           0.96       300
   macro avg       0.89      0.91      0.90       300
weighted avg       0.96      0.96      0.96       300



In [47]:
models = [
    (
        "Logistic Regression",
        LogisticRegression(C=1,solver='liblinear'),
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "Random Forest",
        RandomForestClassifier(n_estimators=30, max_depth=3),
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier",
        XGBClassifier( use_label_encoder = False , eval_metric='logloss'),
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier With SMOTE",
        XGBClassifier( use_label_encoder = False , eval_metric='logloss'),
        (X_train_res, y_train_res),
        (X_test, y_test)
    )
]

In [48]:
reports = []

for model_name , model , train_set , test_set in models:
    X_train = train_set[0]
    y_train = train_set[1]
    X_test = test_set[0]
    y_test = test_set[1]

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)
    

In [49]:
reports

[{'0': {'precision': 0.9454545454545454,
   'recall': 0.9629629629629629,
   'f1-score': 0.9541284403669725,
   'support': 270.0},
  '1': {'precision': 0.6,
   'recall': 0.5,
   'f1-score': 0.5454545454545454,
   'support': 30.0},
  'accuracy': 0.9166666666666666,
  'macro avg': {'precision': 0.7727272727272727,
   'recall': 0.7314814814814814,
   'f1-score': 0.749791492910759,
   'support': 300.0},
  'weighted avg': {'precision': 0.9109090909090909,
   'recall': 0.9166666666666666,
   'f1-score': 0.91326105087573,
   'support': 300.0}},
 {'0': {'precision': 0.96415770609319,
   'recall': 0.9962962962962963,
   'f1-score': 0.9799635701275046,
   'support': 270.0},
  '1': {'precision': 0.9523809523809523,
   'recall': 0.6666666666666666,
   'f1-score': 0.7843137254901961,
   'support': 30.0},
  'accuracy': 0.9633333333333334,
  'macro avg': {'precision': 0.9582693292370712,
   'recall': 0.8314814814814815,
   'f1-score': 0.8821386478088503,
   'support': 300.0},
  'weighted avg': {'prec

In [58]:
mlflow.set_tracking_uri("http://127.0.0.1:5000/")
mlflow.set_experiment('Anomaly Detection')

for i, element in enumerate(models):
    model_name = element[0]
    model = element[1]
    report = reports[i]
    
    with mlflow.start_run(run_name=model_name):
        mlflow.log_param('model_name', model_name)
        mlflow.log_metric('accuracy',report['accuracy'])
        mlflow.log_metric('recall_class_1',report['1']['recall'])
        mlflow.log_metric('recall_class_0',report['0']['recall'])
        mlflow.log_metric('f1_score_macro',report['macro avg']['f1-score'])
        
        if "XGB" in model_name:
            mlflow.xgboost.log_model(model, "model")
        else:
            mlflow.sklearn.log_model(model, "model")

2025/01/30 15:50:24 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Logistic Regression at: http://127.0.0.1:5000/#/experiments/842107654297574889/runs/1d4586cf7fda435b9a4e8c6ec546aeed
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/842107654297574889


2025/01/30 15:51:44 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Random Forest at: http://127.0.0.1:5000/#/experiments/842107654297574889/runs/665c5c8c0b704a198b0e778d44ded6c6
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/842107654297574889


2025/01/30 15:53:01 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run XGBClassifier at: http://127.0.0.1:5000/#/experiments/842107654297574889/runs/9b2e93e211434043bc9bd4f0f2e76263
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/842107654297574889


2025/01/30 15:54:18 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run XGBClassifier With SMOTE at: http://127.0.0.1:5000/#/experiments/842107654297574889/runs/7c44ce78cd284587b6e81a8e8459cfe5
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/842107654297574889


In [55]:
for i, element in enumerate(models):
    model_name = element[0]
    model = element[1]
    report = reports[i]
    print(model_name)
    print(model)
    print(report)

Logistic Regression
LogisticRegression(C=1, solver='liblinear')
{'0': {'precision': 0.9454545454545454, 'recall': 0.9629629629629629, 'f1-score': 0.9541284403669725, 'support': 270.0}, '1': {'precision': 0.6, 'recall': 0.5, 'f1-score': 0.5454545454545454, 'support': 30.0}, 'accuracy': 0.9166666666666666, 'macro avg': {'precision': 0.7727272727272727, 'recall': 0.7314814814814814, 'f1-score': 0.749791492910759, 'support': 300.0}, 'weighted avg': {'precision': 0.9109090909090909, 'recall': 0.9166666666666666, 'f1-score': 0.91326105087573, 'support': 300.0}}
Random Forest
RandomForestClassifier(max_depth=3, n_estimators=30)
{'0': {'precision': 0.96415770609319, 'recall': 0.9962962962962963, 'f1-score': 0.9799635701275046, 'support': 270.0}, '1': {'precision': 0.9523809523809523, 'recall': 0.6666666666666666, 'f1-score': 0.7843137254901961, 'support': 30.0}, 'accuracy': 0.9633333333333334, 'macro avg': {'precision': 0.9582693292370712, 'recall': 0.8314814814814815, 'f1-score': 0.8821386478

In [65]:
model_name = "XGB-Smote"
run_id = input("Enter the run_id: ")


model_uri = f"runs:/{run_id}/model"
result = mlflow.register_model(
    model_uri , model_name
    # "runs:/d16076a3ec534311817565e6527539c0/sklearn-model", "sk-learn-random-forest-reg"
)

Successfully registered model 'XGB-Smote'.
2025/01/30 16:45:54 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: XGB-Smote, version 1
Created version '1' of model 'XGB-Smote'.


In [61]:
model_version = 1
model_uri = f"models:/{model_name}/{model_version}"

loaded_model = mlflow.xgboost.load_model(model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:4]

MlflowException: The following failures occurred while downloading one or more artifacts from http://127.0.0.1:5000/api/2.0/mlflow-artifacts/artifacts/842107654297574889/7c44ce78cd284587b6e81a8e8459cfe5/artifacts/XGB-Smote:
##### File  #####
API request to http://127.0.0.1:5000/api/2.0/mlflow-artifacts/artifacts/842107654297574889/7c44ce78cd284587b6e81a8e8459cfe5/artifacts/XGB-Smote/ failed with exception HTTPConnectionPool(host='127.0.0.1', port=5000): Max retries exceeded with url: /api/2.0/mlflow-artifacts/artifacts/842107654297574889/7c44ce78cd284587b6e81a8e8459cfe5/artifacts/XGB-Smote/ (Caused by ResponseError('too many 500 error responses'))

In [66]:
model_uri = f"models:/{model_name}@test"
model_name = "XGB-Smote"
model_version = 2

loaded_model = mlflow.xgboost.load_model(model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:4]

array([0, 0, 0, 0])

In [67]:
dev_model_uri = f"models:/{model_name}@test"
prod_model = 'anomaly-detection-0404prod'

client = mlflow.tracking.MlflowClient()
client.copy_model_version(src_model_uri=dev_model_uri ,dst_name=prod_model)

Successfully registered model 'anomaly-detection-0404prod'.
Copied version '1' of model 'XGB-Smote' to version '1' of model 'anomaly-detection-0404prod'.


<ModelVersion: aliases=[], creation_timestamp=1738230416877, current_stage='None', description='', last_updated_timestamp=1738230416877, name='anomaly-detection-0404prod', run_id='7c44ce78cd284587b6e81a8e8459cfe5', run_link='', source='models:/XGB-Smote/1', status='READY', status_message=None, tags={}, user_id='', version='1'>